In [1]:
import pandas  as pd
from langchain.llms import OpenAI
from dotenv import load_dotenv

import openai,os
load_dotenv(r'D:\Git\NLP\LLM\ActivLoop\.env')
openai_api_key = os.getenv("ACTIVELOOP_TOKEN")

assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key

openai.api_base = os.getenv("OpenAiService")
openai.api_type = "azure"
openai.api_version =os.getenv("OpenAiVersion")
davincimodel= os.getenv("OpenAiDavinci")
active_loop_token=os.getenv("ACTIVELOOP_TOKEN")
embedding_model=os.getenv("OpenAiEmbedding")
chat_ai=os.getenv("ChatAI")#
HUGGINGFACEHUB_API_TOKEN=os.getenv("HUGGINGFACEHUB_API_TOKEN")

c:\Users\sri.karan\.conda\envs\activeloop\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.17) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In this lesson, we are going to delve into a use case of AI technology in the creative domain of children's picture book creation in a project called "FableForge", leveraging both OpenAI GPT-3.5 LLM for writing the story and Stable Diffusion for generating images for it.

### Introduction
This lesson’s project, FableForge, is an application that generates picture books from a single text prompt. It utilizes the power of OpenAI's language model GPT-3.5 to write the story. Then, the text is transformed into visual prompts for Stable Diffusion, an AI that creates corresponding images, resulting in a complete picture book. The data, both text and images, are then stored in a Deep Lake dataset for easy analysis.

The article guides us through the steps of building FableForge, detailing the challenges, successes, and methodologies adopted. You will learn how the team leveraged the “function calling” feature newly introduced by OpenAI, which is used in this project specifically to structure text data suitable for Stable Diffusion, a task that initially proved difficult due to the model's tendency to include non-visual content in the prompts. We’ll see how to overcome this by using a function providing structured, actionable output for external tools.

We'll delve into each component of FableForge, including the generation of text and images, combining them into a book format, storing the data into Deep Lake, and finally presenting it all through a user-friendly interface with Streamlit. We'll explore the process of text generation, extracting visual prompts, assembling PDFs, and uploading the data to Deep Lake.

By the end of this lesson, you'll gain a comprehensive understanding of how various AI tools and methodologies can be effectively integrated to overcome challenges and open new frontiers in creative domains.

In [2]:
BOOK_TEXT_PROMPT = """
Write an engaging, great 3-6 page children's picture book. Each page should have 2-3 sentences. There should be rhymes.
We will be adding pictures of the environment/scenery for each page, so pick a pretty setting/place. Limit of 7 pages,
do not exceed 3 sentences per page. Do not exceed 7 pages.

Before the story begins, write a "Page 0: {title}" page. The title should be the name of the book, no more than four words.


Format like: Page 0: {title}, Page 1: {text}, etc. Do not write anything else. 
"""


get_visual_description_function = [{
    'name': 'get_passage_setting',
    'description': 'Generate and describe the visuals of a passage in a book. Visuals only, no characters, plot, or people. Highly detailed',
    'parameters': {
        'type': 'object',
        'properties': {
            'base_setting': {
                'type': 'string',
                'description': 'The base setting of the passage, e.g. ancient Rome, Switzerland, etc.',
            },
            'setting': {
                'type': 'string',
                'description': 'The detailed visual setting of the passage, e.g. a a snowy mountain village',
            },
            'time_of_day': {
                'type': 'string',
                'description': 'The detailed time of day of the passage, e.g. nighttime, daytime, dawn.',
            },
            'weather': {
                'type': 'string',
                'description': 'The detailed weather of the passage, eg. heavy rain with dark clouds.',
            },
            'key_elements': {
                'type': 'string',
                'description': 'The detailed key visual elements of the passage, eg colorful houses, a church, and a lake. ',
            },
            'specific_details': {
                'type': 'string',
                'description': 'The detailed specific visual details of the passage, eg lake reflecting the sky.',
            }
        },
        'required': ['base_setting', 'setting', 'time_of_day', 'weather', 'key_elements', 'specific_details']
    }
}]

get_lighting_and_atmosphere_function = [{
    'name': 'get_lighting_and_atmosphere',
    'description': 'Generate a  highly detailed visual description of the overall atmosphere and color palette of a book',
    'parameters': {
        'type': 'object',
        'properties': {
            'lighting': {
                'type': 'string',
                'description': 'The lighting atmosphere of the book, eg. cheerful atmosphere',
            },
            'mood': {
                'type': 'string',
                'description': 'The mood of the book, eg. lively mood',
            },
            'color_palette': {
                'type': 'string',
                'description': 'The color palette of the book, eg. bright and vivid color palette',
            },
        },
        'required': ['lighting', 'mood', 'color_palette']
    }
}]

In [3]:
import sys

In [4]:
sys.path.append(r"c:\users\sri.karan\appdata\roaming\python\python38\site-packages")

https://www.activeloop.ai/resources/ai-story-generator-open-ai-function-calling-lang-chain-stable-diffusion/

In [5]:
import re
from concurrent.futures import ThreadPoolExecutor

from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
import replicate

import json


# from prompts import *

import requests

import streamlit as st





In [6]:
class BuildBook:
    book_text_prompt = BOOK_TEXT_PROMPT

    def __init__(self, model_name, input_text, style):
        self.chat = ChatOpenAI(engine=chat_ai)
        self.input_text = input_text
        self.style = style

        self.progress = st.progress(0)
        self.progress_steps = 0
        self.total_progress_steps = 30

        self.progress_steps += 2
        self.progress.progress(self.progress_steps / self.total_progress_steps, "Generating book text...")
        self.book_text = self.get_pages()

        self.progress_steps += 2
        self.progress.progress(self.progress_steps / self.total_progress_steps, "Generating SD prompts...")

        self.pages_list = self.get_list_from_text(self.book_text)

        self.sd_prompts_list = self.get_prompts()


        self.source_files = self.download_images()
        self.list_of_tuples = self.create_list_of_tuples()
        self.progress.progress(1.0, "Done! Wait one moment while your book is processed...")



    def get_pages(self):
        pages = self.chat([HumanMessage(content=f'{self.book_text_prompt} Topic: {self.input_text}')]).content
        return pages

    def get_prompts(self):
        base_atmosphere = self.chat([HumanMessage(content=f'Generate a visual description of the overall lightning/atmosphere of this book using the function.'
                                                          f'{self.book_text}')], functions=get_lighting_and_atmosphere_function)
        base_dict = func_json_to_dict(base_atmosphere)

        summary = self.chat([HumanMessage(content=f'Generate a concise summary of the setting and visual details of the book')]).content

        base_dict['summary_of_book_visuals'] = summary

        def generate_prompt(page, base_dict):
            prompt = self.chat([HumanMessage(content=f'General book info: {base_dict}. General style: {self.style} Passage: {page}.'
                                                     f' Generate a visual description of the passage using the function.'
                                                     f'Creatively fill all parameters with guessed/assumed values if they are missing.')],
                               functions=get_visual_description_function)
            return func_json_to_dict(prompt)

        with ThreadPoolExecutor(max_workers=10) as executor:
            prompt_list = list(executor.map(generate_prompt, self.pages_list, [base_dict] * len(self.pages_list)))

        prompts = prompt_combiner(prompt_list, base_dict, self.style)


        return prompts

    def get_list_from_text(self, text):
        new_list = re.split('Page \d+:', text)
        new_list.pop(0)
        return new_list

    def create_images(self):
        if len(self.pages_list) != len(self.sd_prompts_list):
            raise 'Pages and Prompts do not match'

        def generate_image(i, prompt):
            print(f'{prompt} is the prompt for page {i + 1}')
            output = replicate.run(
                "stability-ai/stable-diffusion:db21e45d3f7023abc2a46ee38a23973f6dce16bb082a930b0c49861f96d1e5bf",
                input={"prompt": 'art,' + prompt,
                       "negative_prompt": "photorealistic, photograph, bad anatomy, blurry, gross,"
                                          "weird eyes, creepy, text, words, letters, realistic"
                                          },
            )
            return output[0]

        with ThreadPoolExecutor(max_workers=10) as executor:
            image_urls = list(executor.map(generate_image, range(len(self.sd_prompts_list)), self.sd_prompts_list))

        return image_urls

    def download_images(self):
        image_urls = self.create_images()
        source_files = []
        for i, url in enumerate(image_urls):
            r = requests.get(url, stream=True)
            file_path = f'images/{i+1}.png'
            with open(file_path, 'wb') as file:
                source_files.append(file_path)
                for chunk in r.iter_content():
                    file.write(chunk)
                self.progress_steps += 1
                self.progress.progress(self.progress_steps / self.total_progress_steps, f"Downloading image {i+1}...")
        return source_files

    def create_list_of_tuples(self):
        files = self.source_files
        text = self.pages_list
        return list(zip(files, text))


def func_json_to_dict(response):
    return json.loads(response.additional_kwargs['function_call']['arguments'])


def prompt_combiner(prompt_list, base_dict, style):
    prompts = []
    for i, prompt in enumerate(prompt_list):
        entry = f"{prompt['base_setting']}, {prompt['setting']}, {prompt['time_of_day']}, {prompt['weather']}, {prompt['key_elements']}, {prompt['specific_details']}, " \
                f"{base_dict['lighting']}, {base_dict['mood']}, {base_dict['color_palette']}, in the style of {style}"
        prompts.append(entry)
    return prompts

def process_page(chat, page, base_dict):
    prompt = chat([HumanMessage(content=f'General book info: {base_dict}. Passage: {page}')],
                      functions=get_visual_description_function)
    return func_json_to_dict(prompt)


https://github.com/e-johnstonn/FableForge/tree/master